Installare tutti i pacchetti necessari e scaricare il modello it_core_news_md (italiano, dimensione media) di spacy

In [ ]:
!pip install gensim spacy numpy torch networkx scikit-learn
!python -m spacy download it_core_news_md


Scaricare gli embeddings di fast text, saranno utili per la creazione di vettori a partire dalle parole. I vettori saranno utili per calcolare successivamente la similarità coseno e per eseguire una clusterizzazione dei concetti nel metodo extract_concepts_clustering. Gli embedding sono scaricati in formato binario perchè richiede meno risorse per la lettura (il pacchetto è più pesante però).

In [ ]:
# Esempio di URL diretto (verifica sempre l'URL corretto sul sito ufficiale)
url = 'https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.it.300.bin.gz'

# Scarica il file
!wget -O cc.it.300.bin.gz $url

# Estrai il file compresso
!gunzip cc.it.300.bin.gz


--2024-11-19 17:50:50--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.it.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.25, 13.226.210.78, 13.226.210.15, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4502592726 (4.2G) [application/octet-stream]
Saving to: ‘cc.it.300.bin.gz’

cc.it.300.bin.gz    100%[===================>]   4.19G  45.0MB/s    in 65s     

2024-11-19 17:51:55 (66.3 MB/s) - ‘cc.it.300.bin.gz’ saved [4502592726/4502592726]

gzip: cc.it.300.bin already exists; do you wish to overwrite (y or n)? y
y


Scaricare fasttext

In [ ]:
!pip install fastText

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fastText: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4296182 sha256=66d7d00e464792b2c6ec253d530f49f8b3c8b11afc2040c68a10edd9f2c2acd4
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fastText


Nella cella successiva c'è un semplice controllo, utilizzando il modulo OS, per verificare la presenza degli embeddings.

In [ ]:

import os

fasttext_path = '/content/cc.it.300.bin'

if os.path.exists(fasttext_path):
    print("Il file FastText è stato scaricato e decompresso correttamente.")
else:
    print("Errore nel download o nella decompressione del file FastText.")


Il file FastText è stato scaricato e decompresso correttamente.


In questa cella, oltre a importare tutti i moduli necessari per le classi successive, viene caricato il modello spacy e vengono caricati gli embedding attraverso il metodo load_model di fasttext.

In [ ]:
import spacy
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import networkx as nx
from fasttext import load_model
from sklearn.cluster import KMeans
import pickle
import os
import plotly.graph_objs as go
print("SpaCy usa la GPU:", spacy.prefer_gpu())
# -------------------------------
# Caricamento del Modello Linguistico e degli Embedding
# -------------------------------

# Percorso al modello FastText binario
fasttext_path = '/content/cc.it.300.bin'

# Caricamento del modello di Spacy
print("Caricamento del modello SpaCy...")
spacy_model = spacy.load("it_core_news_md")
print("Modello 'it_core_news_md' caricato con successo.")

# Caricamento degli embedding FastText
print("Caricamento degli embedding FastText...")
try:
    fasttext_model = load_model(fasttext_path)
    print("Embedding FastText caricati con successo.")
except Exception as e:
    print(f"Errore durante il caricamento del modello FastText: {e}")

SpaCy usa la GPU: True
Caricamento del modello SpaCy...
Modello 'it_core_news_md' caricato con successo.
Caricamento degli embedding FastText...
Embedding FastText caricati con successo.


In questa cella, oltre a caricare altri modulo utili per la gestione del grafo, creiamo una funzione a livello globale per calcolare la similarità tra vettori, che verrà successivamente chiamata. Successivamente creiamo le classi di cui abbiamo bisogno per creare il grafo di conoscenza (ThreeDimensionalGraphModel) ed estrarre concetti dalla frase input attraverso l'uso del modello NLP (classe TextToGraph)

In [ ]:
import networkx as nx
import pickle
import numpy as np
from sklearn.cluster import KMeans

# Funzione per calcolare la similarità coseno
def calculate_similarity(word1, word2, model):
    try:
        vec1 = model.get_word_vector(word1)
        vec2 = model.get_word_vector(word2)
        similarity = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
        return similarity
    except KeyError:
        print(f"Una delle parole '{word1}' o '{word2}' non è nel modello FastText.")
        return 0.0

# Classe ThreeDimensionalGraphModel
class ThreeDimensionalGraphModel:
    def __init__(self):
        self.G = nx.Graph()
        self.C = {}  # Dizionario di associazione concetti-nodi
        self.truth_node = 'Verità'
        self.G.add_node(self.truth_node)

    def add_connection(self, node_i, node_j, weight=1, probability=1.0, relation_type='default'):
        self.G.add_node(node_i)
        self.G.add_node(node_j)
        if self.G.has_edge(node_i, node_j):
            self.G[node_i][node_j]['weight'] += weight
            self.G[node_i][node_j]['probability'] = (
                self.G[node_i][node_j]['probability'] + probability
            ) / 2
            if 'relation_types' not in self.G[node_i][node_j]:
                self.G[node_i][node_j]['relation_types'] = set()
            self.G[node_i][node_j]['relation_types'].add(relation_type)
        else:
            self.G.add_edge(node_i, node_j, weight=weight, probability=probability, relation_types={relation_type})

    def add_concept_association(self, concept, node, strength=1.0):
        if concept not in self.C:
            self.C[concept] = {}
        if node not in self.C[concept]:
            self.C[concept][node] = 0
        self.C[concept][node] += strength

    def save_state(self, filename):
        with open(filename + '_G.pkl', 'wb') as f:
            pickle.dump(self.G, f)
        with open(filename + '_C.pkl', 'wb') as f:
            pickle.dump(self.C, f)
        print("Stato del grafo e delle associazioni salvati.")

    def load_state(self, filename):
        with open(filename + '_G.pkl', 'rb') as f:
            self.G = pickle.load(f)
        with open(filename + '_C.pkl', 'rb') as f:
            self.C = pickle.load(f)
        print("Stato del grafo e delle associazioni caricato.")

# Classe TextToGraph
class TextToGraph:
    def __init__(self):
        self.graph_model = ThreeDimensionalGraphModel()
        self.node_dict = {}
        self.reverse_node_dict = {}
        self.concept_dict = {}
        self.reverse_concept_dict = {}
        self.text = ""

    def process_text(self, text, nlp):
        self.text = text
        doc = nlp(text)
        for sent in doc.sents:
            subject, obj = None, None
            relation = 'default'
            for token in sent:
                if not token.is_stop and not token.is_punct:
                    if "subj" in token.dep_:
                        subject = token.lemma_.lower()
                    elif "obj" in token.dep_:
                        obj = token.lemma_.lower()
                    elif token.pos_ == 'VERB':
                        relation = token.lemma_.lower()
            if subject and obj:
                subj = self._get_node(subject)
                obj = self._get_node(obj)
                probability = self._calculate_probability(subject, obj)
                self.graph_model.add_connection(subj, obj, weight=1, probability=probability, relation_type=relation)

    def _get_node(self, name):
        if name not in self.node_dict:
            new_index = len(self.node_dict)
            self.node_dict[name] = new_index
            self.reverse_node_dict[new_index] = name
        return name

    def _calculate_probability(self, subject, obj):
        if fasttext_model:
            similarity = calculate_similarity(subject, obj, fasttext_model)
            probability = (similarity + 1) / 2
            return probability
        else:
            return 0.5

    def extract_concepts_clustering(self, num_concepts=10):
        node_embeddings = []
        node_names = []

        for node in self.node_dict:
            try:
                embedding_vector = fasttext_model.get_word_vector(node)
            except KeyError:
                embedding_vector = np.random.randn(300)  # Supponendo dimensione 300
            node_embeddings.append(embedding_vector)
            node_names.append(node)

        node_embeddings = np.array(node_embeddings)
        clustering_model = KMeans(n_clusters=num_concepts, random_state=42)
        labels = clustering_model.fit_predict(node_embeddings)
        centroids = clustering_model.cluster_centers_

        for concept_label in range(num_concepts):
            cluster_indices = np.where(labels == concept_label)[0]
            if len(cluster_indices) == 0:
                continue
            cluster_embeddings = node_embeddings[cluster_indices]
            distances = np.linalg.norm(cluster_embeddings - centroids[concept_label], axis=1)
            representative_idx = cluster_indices[np.argmin(distances)]
            representative_name = node_names[representative_idx]

            if representative_name not in self.concept_dict:
                new_concept_idx = len(self.concept_dict)
                self.concept_dict[representative_name] = new_concept_idx
                self.reverse_concept_dict[new_concept_idx] = representative_name
            else:
                new_concept_idx = self.concept_dict[representative_name]

            for idx in cluster_indices:
                node = node_names[idx]
                self.graph_model.add_concept_association(new_concept_idx, node, strength=1.0)

    def save_state(self, filename):
        with open(filename + '_node_dict.pkl', 'wb') as f:
            pickle.dump(self.node_dict, f)
        with open(filename + '_reverse_node_dict.pkl', 'wb') as f:
            pickle.dump(self.reverse_node_dict, f)
        with open(filename + '_concept_dict.pkl', 'wb') as f:
            pickle.dump(self.concept_dict, f)
        with open(filename + '_reverse_concept_dict.pkl', 'wb') as f:
            pickle.dump(self.reverse_concept_dict, f)
        self.graph_model.save_state(filename)

    def load_state(self, filename):
        with open(filename + '_node_dict.pkl', 'rb') as f:
            self.node_dict = pickle.load(f)
        with open(filename + '_reverse_node_dict.pkl', 'rb') as f:
            self.reverse_node_dict = pickle.load(f)
        with open(filename + '_concept_dict.pkl', 'rb') as f:
            self.concept_dict = pickle.load(f)
        with open(filename + '_reverse_concept_dict.pkl', 'rb') as f:
            self.reverse_concept_dict = pickle.load(f)
        self.graph_model.load_state(filename)

    def print_nodes_and_connections(self):
        """
        Stampa i nodi e le loro connessioni con peso, probabilità e tipo di relazione.
        """
        print("\n--- Nodi e Connessioni ---")
        G = self.graph_model.G
        if G is not None and G.number_of_edges() > 0:
            for edge in G.edges(data=True):
                source, target, data = edge
                weight = data.get('weight', 1)
                probability = data.get('probability', 1.0)
                relation_types = data.get('relation_types', {'default'})
                print(f"Connessione da '{source}' a '{target}' con peso {weight}, probabilità {probability:.2f}, relazione: {relation_types}")
        else:
            print("Nessuna connessione trovata.")


In questa cella creiamo una classe per visulizzare il grafo.

In [ ]:

import plotly.graph_objs as go

def visualize_graph_3d(graph_model):
    """
    Visualizza il grafo in 3D utilizzando Plotly.
    """
    # Calcolo del layout 3D
    pos = nx.spring_layout(graph_model.G, dim=3, seed=42)
    edge_x = []
    edge_y = []
    edge_z = []

    # Aggiunge le coordinate degli archi
    for edge in graph_model.G.edges():
        x0, y0, z0 = pos[edge[0]]
        x1, y1, z1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    # Traccia degli archi
    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        line=dict(width=1, color='blue'),
        hoverinfo='none',
        mode='lines'
    )

    node_x = []
    node_y = []
    node_z = []
    node_text = []

    # Aggiunge le coordinate dei nodi
    for node in graph_model.G.nodes():
        x, y, z = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_z.append(z)
        node_text.append(node)  # Nome del nodo

    # Traccia dei nodi
    node_trace = go.Scatter3d(
        x=node_x, y=node_y, z=node_z,
        mode='markers',
        marker=dict(
            size=10,
            color='red',
            opacity=0.8
        ),
        text=node_text,
        hoverinfo='text'
    )

    # Configura la visualizzazione
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='Grafo di Conoscenza 3D',
                        showlegend=False,
                        scene=dict(
                            xaxis=dict(showbackground=False),
                            yaxis=dict(showbackground=False),
                            zaxis=dict(showbackground=False)
                        ),
                        margin=dict(l=0, r=0, b=0, t=40)
                    ))
    fig.show()



Questa cella serve per eseguire dei debug e per salvare o caricare il grafo in un formato pkl

In [ ]:
import pickle
import networkx as nx

# Carica il grafo salvato
with open("modello_salvato_G.pkl", "rb") as f:
    graph = pickle.load(f)

# Stampa informazioni sul grafo
print("Informazioni sul Grafo:")

# Visualizza i nodi e gli archi
print("\nNodi nel grafo:")
print(graph.nodes(data=True))

print("\nArchi nel grafo:")
print(graph.edges(data=True))

# Carica le associazioni concetto-nodo
with open("modello_salvato_C.pkl", "rb") as f:
    concept_associations = pickle.load(f)

# Stampa le associazioni
print("\nAssociazioni Concetto-Nodo:")
for concept, nodes in concept_associations.items():
    print(f"Concetto '{concept}':")
    for node, strength in nodes.items():
        print(f"  Nodo: {node}, Forza: {strength}")

# Carica il dizionario dei nodi
with open("modello_salvato_node_dict.pkl", "rb") as f:
    node_dict = pickle.load(f)

# Carica il dizionario inverso dei nodi
with open("modello_salvato_reverse_node_dict.pkl", "rb") as f:
    reverse_node_dict = pickle.load(f)

# Stampa i dizionari
print("\nDizionario dei Nodi:")
print(node_dict)

print("\nDizionario Inverso dei Nodi:")
print(reverse_node_dict)

# Carica il dizionario dei concetti
with open("modello_salvato_concept_dict.pkl", "rb") as f:
    concept_dict = pickle.load(f)

# Carica il dizionario inverso dei concetti
with open("modello_salvato_reverse_concept_dict.pkl", "rb") as f:
    reverse_concept_dict = pickle.load(f)

# Stampa i dizionari
print("\nDizionario dei Concetti:")
print(concept_dict)

print("\nDizionario Inverso dei Concetti:")
print(reverse_concept_dict)


Nell'ultima cella definiamo la main function e istanziamo gli oggetti per testare il tutto.

In [ ]:
def main():
    # Nome base per i file di salvataggio
    save_filename = 'modello_salvato'

    # Testo complesso di esempio
    text = """
Nel XVII secolo, Galileo Galilei introdusse il metodo scientifico, rivoluzionando la comprensione dell'universo e affrontando l'opposizione della Chiesa cattolica.
Le sue osservazioni delle fasi di Venere e delle lune di Giove supportarono il modello eliocentrico di Copernico, segnando l'inizio della scienza moderna.

La fotosintesi, un processo scoperto da Jan Ingenhousz nel XVIII secolo, converte l'energia solare in glucosio, con l'ossigeno come sottoprodotto.
Questo processo è essenziale per mantenere l'equilibrio dei gas atmosferici e sostenere la vita sulla Terra.

Nel XX secolo, Alan Turing sviluppò i concetti fondamentali dell'intelligenza artificiale, introducendo l'idea di una macchina capace di simulare il pensiero umano.
Oggi, gli algoritmi di machine learning vengono utilizzati per analizzare grandi quantità di dati, con applicazioni nella medicina, nella finanza e nella robotica.

Il cambio climatico, intensificato dall'aumento delle emissioni di CO2, sta causando l'innalzamento del livello del mare, scioglimento dei ghiacciai e eventi meteorologici estremi.
L'accordo di Parigi del 2015 ha cercato di unire le nazioni per limitare l'aumento della temperatura globale a 1.5°C rispetto ai livelli preindustriali.

Filosoficamente, Immanuel Kant sosteneva che il tempo e lo spazio fossero forme a priori della conoscenza, mentre Friedrich Nietzsche dichiarava la "morte di Dio", esplorando le conseguenze morali e culturali di un mondo senza valori assoluti.

In biologia molecolare, la scoperta della doppia elica del DNA da parte di Watson e Crick ha fornito una comprensione della genetica, mentre le mutazioni geniche sono state identificate come la base dell'evoluzione e di molte malattie.
La CRISPR-Cas9, una tecnica innovativa di editing genetico, permette oggi di modificare il genoma con una precisione senza precedenti.

In astrofisica, la scoperta delle onde gravitazionali da parte del team di LIGO nel 2015 ha confermato un'altra previsione della teoria della relatività generale di Einstein, aprendo nuove finestre sull'universo.
Queste onde, causate dalla fusione di buchi neri o stelle di neutroni, trasportano informazioni sulla struttura dello spazio-tempo.

La civiltà mesopotamica, sviluppatasi tra i fiumi Tigri ed Eufrate, introdusse la scrittura cuneiforme, che permise la registrazione di leggi, transazioni commerciali e opere letterarie come l'Epopea di Gilgamesh.
Nel frattempo, gli Egizi costruirono le piramidi, utilizzando avanzate tecniche di ingegneria ancora oggetto di studio.

Un robot può oggi esplorare Marte, analizzare campioni di suolo e comunicare dati scientifici alla Terra, mentre una formica utilizza feromoni per orientarsi nel suo ambiente.
L'intelligenza collettiva di un alveare rappresenta un modello biologico per la progettazione di algoritmi distribuiti.

In letteratura, William Shakespeare esplorò la complessità dell'animo umano in opere come "Amleto" e "Macbeth", mentre Dante Alighieri, con la "Divina Commedia", descrisse un viaggio allegorico attraverso Inferno, Purgatorio e Paradiso.


    """


    # Inizializzazione del modello TextToGraph
    text_to_graph = TextToGraph()

    # Carica lo stato precedente, se esiste
    if os.path.exists(save_filename + '_G.pkl'):
        text_to_graph.load_state(save_filename)
        print("Stato precedente caricato.")
    else:
        print("Nessun stato precedente trovato. Creazione di un nuovo modello.")

    # Processa il testo e aggiorna il grafo
    print("\nProcesso del testo...")
    text_to_graph.process_text(text, spacy_model)

    # Stampa nodi e connessioni
    print("\n--- Nodi e Connessioni ---")
    text_to_graph.print_nodes_and_connections()

    # Salva lo stato aggiornato
    text_to_graph.save_state(save_filename)
    print("\nStato aggiornato salvato correttamente.")

if __name__ == "__main__":
    main()


Stato del grafo e delle associazioni caricato.
Stato precedente caricato.

Processo del testo...

--- Nodi e Connessioni ---

--- Nodi e Connessioni ---
Connessione da 'osservazione' a 'inizio' con peso 2, probabilità 0.68, relazione: {'segnare'}
Connessione da 'processo' a 'equilibrio' con peso 2, probabilità 0.64, relazione: {'sostenere'}
Connessione da 'alan' a 'pensiero' con peso 2, probabilità 0.52, relazione: {'simulare'}
Connessione da 'algoritmi' a 'quantità' con peso 2, probabilità 0.58, relazione: {'analizzare'}
Connessione da 'cambio' a 'innalzamento' con peso 2, probabilità 0.71, relazione: {'causare'}
Connessione da 'accordo' a 'aumento' con peso 2, probabilità 0.63, relazione: {'limitare'}
Connessione da 'friedrich' a 'conseguenza' con peso 2, probabilità 0.56, relazione: {'esplorare'}
Connessione da 'mutazione' a 'comprensione' con peso 2, probabilità 0.66, relazione: {'identificare'}
Connessione da 'crispr-cas9' a 'genoma' con peso 2, probabilità 0.62, relazione: {'modi

Come si vede dall'output vengono correttamente create all'interno del grafo le  